In [1]:
from sklearn.model_selection import train_test_split
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import os
from scipy import stats
from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import RobustScaler
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import QuantileTransformer
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
from sklearn.metrics import r2_score
from sklearn.linear_model import LassoLars
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import TweedieRegressor
import wrangle

ModuleNotFoundError: No module named 'wrangle'

In [ ]:
df = pd.read_csv('final_df.csv',index_col=[0])

In [ ]:
df

In [ ]:
train, validate, test = wrangle.split_data(df)

In [ ]:
def univariate_visual(df):
    '''
    creates histplots for all of my columns
    '''
    plt.figure(figsize=(25,15))
    plt.xticks(rotation = 45)
    for i, col in enumerate(df):

        plt.title(col)
        sns.histplot(df[col])
        plt.xticks(rotation=45)
        plt.show()
        
    plt.show()

In [ ]:
univariate_visual(train)

In [ ]:
def correlation_charts(train,columns_list, target):
    '''
    Creates and shows visuals for Correlation tests 
    '''
    plt.figure(figsize=(14,3))
    plt.suptitle('Bivariate Exploration: The Strongest Correlators of Wine Quality')
    for i, col in enumerate(train[columns_list]):
        if col != target:

            sns.regplot(data = train, x = col, y = target, scatter_kws={'alpha': 0.1}, line_kws={'color': 'red'})

            plt.show()

In [ ]:
columns_list = df.select_dtypes(exclude=['object']).columns.to_list()

In [ ]:
target = columns_list.pop(len(columns_list) -1 )

In [ ]:
target

time to throw has a neutral correlation towards our target variable
avg comp air yards has a slightly negative correlation towards our target variable
avt att air yards has a negative correlation towards our target variable
avg air yard diff has a positive correlation towards our target variable
aggressive has a negative correlation towards our target variable
longest comp air dist has a positive correlation towards our target variable
air yards to sticks has a slightly negative correlation towards our target variable
pass attempts have a strong positive correlation towards our target variable
total yards has a strong positive correlation towards our target variable
touchdowns has a strong positive correlation towards our target variable
interceptions has a positive correlation towards our target variable
pass rating has a positive correlation towards our target variable
completion percentage has a positive correlation towars our target variable
exp completion percentage has a positive correlation towards our target variable
completion pct abv exp has a positive correlation towards our target variable
year has a neutral correlation with out target variable
first round has a slightly positive correlation with our target variable
div round has a slightly positive correlation with our target variable
conference champ has a slightly positive correlation with our target variable
superbowl has a neutral correlation with our target variable
won superbowl has a slightly positive correlation with our target variable

In [ ]:
columns_list.append('win')

In [ ]:
correlation_charts(train,columns_list, target)

In [ ]:
def correlation_tests(train, columns_list, target):
    '''
    Runs a correlation test on dataframe features vs target variable
    '''
    corr_df = pd.DataFrame({'feature': [],
                        'r': [],
                       'p': []})
    for i, col in enumerate(train[columns_list]):
        r, p = stats.pearsonr(train[col], train[target])
        corr_df.loc[i] = [col, abs(r), p]
    to_return = corr_df.sort_values(by='r', ascending=False)
    to_return['target'] = target
    return to_return

p value was greater than alpha for:
> - year
> - superbowl
> - avg_comp_air_yds
> - won_superbowl
> - air_yds_to_sticks
> - time_to_throw
> - avg_air_yard_diff
> - avg_att_air_yards
> - conf_champ

- We will use RMSE as our evaluation metric

** by using baseline as an evaluation metric we can be accurate to within 6.9% <br>
** 6.9% will be the baseline RMSE we will use for this project <br>
<br>
** I will be evaluating models developed using four different model types and various hyperparameter configurations * Models will be evaluated on train and validate data * The model that performs the best will then be evaluated on test data

Feature we are moving forward with
> - total_yds
> - pass_att
> - td
> - pass_rating
> - completion_pct
> - exp_completion_pct
> - completion_pct_abv_exp
> - lon_comp_air_dist
> - int
> - div_round
> - aggressive
> - first_round

In [ ]:
correlation_tests(train, columns_list, target)

In [ ]:
X_train, X_validate, X_test, y_train, y_validate, y_test = wrangle.get_X_train_val_test(train,validate, test, columns_list,target)

In [ ]:
def scale_data(train,
               validate,
               test,
               cols = columns_list):
    '''Takes in train, validate, and test set, and outputs scaled versions of the columns that were sent in as dataframes'''
    #Make copies for scaling
    train_scaled = train.copy() #Ah, making a copy of the df and then overwriting the data in .transform() to remove warning message
    validate_scaled = validate.copy()
    test_scaled = test.copy()
    #Initiate scaler, using Min max scaler
    scaler = MinMaxScaler()
    #Fit to train only
    scaler.fit(train[cols])
    #Creates scaled dataframes of train, validate, and test. This will still preserve columns that were not sent in initially.
    train_scaled[cols] = scaler.transform(train[cols])
    validate_scaled[cols] = scaler.transform(validate[cols])
    test_scaled[cols] = scaler.transform(test[cols])

    return train_scaled, validate_scaled, test_scaled


In [ ]:
X_train, X_validate, X_test = wrangle.scale_data(X_train,
               X_validate,
               X_test,
               cols = columns_list)

In [ ]:
df1, df2, df3 = wrangle.get_model_numbers(X_train, X_validate, X_test, y_train, y_validate, y_test)

In [ ]:
df1

In [ ]:
df2